In [ ]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit==0.0.2

# Code description
This code aims at verifying (numerically) the first inequality from Theorem 2 of the paper
> "Last-Iterate Convergence of Optimistic Gradient Method for Monotone
       Variationnal Inequalities".

## Problem setup:
Consider the problem of finding a zero of a monotone Lipschitz operator:
       $$ \text{find } x\in Q \text{ such that } \langle F(x);y-x\rangle \geq 0 \text{ for all }y\in Q$$
where $F$ is monotone and $L$-Lipschitz, and $Q$ is convex and compact.

## Algorithm: 
The past extragradient method is described by two sets of iterates:
$x^k$, $\tilde{x}^k$ where $k$ denotes the iteration counter, as follows:

- Initialize $\tilde{x}^0 = x^0$, and $x^1=\mathrm{Proj}_Q(x^0 - \gamma  F(x^0))$, then run
- for $k=1,...,N-1$:
     $$\tilde{x}^k    = \mathrm{Proj}_Q(x^k - \gamma  F(\tilde{x}^{k-1}))$$
     $$ x^{k+1} = \mathrm{Proj}_Q(x^k - \gamma  F(\tilde{x}^k))$$

## Potential function:
Denotes $p_k := \|x^k - x^{k-1}\|^2 + \| x^k - x^{k-1} - 2 \gamma  (F(x^k) - F(\tilde{x}^{k-1})) \|^2$. The code compute the maximum (i.e., worst-case) value of 
$$\|x^{k+1} - x^* \|^2 + 1/16 \| \tilde{x}^{k} - \tilde{x}^{k-1} \|^2 + A_{k+1} p_{k+1}
- \|x^{k} - x^* \|^2 - 1/16 \| \tilde{x}^{k-1} - \tilde{x}^{k-2} \|^2- A_k p_k$$
when $A_{k+1} = A_k + 1/8$. The expression should always be "$\cdot\leq 0$" for verifying the identity from Theorem 2 (with $\gamma\leq 1/4/L$ and $A_k\geq 4/3$). In the code below, we use $k = 2$ for notational convenience.

In [2]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP
from PEPit.operators import LipschitzStronglyMonotoneOperator
from PEPit.functions import ConvexIndicatorFunction
from PEPit.primitive_steps import proximal_step

In [3]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = 1;
gamma = 1/4/L;
A2 = 100000; #too large values might make the solver bug

##########################################################

# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;

# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; mu = 0; # F is 1-Lipschitz and 0-strongly monotone
F    = problem.declare_function(LipschitzStronglyMonotoneOperator, param={'L': L, 'mu': mu})
indQ = problem.declare_function(ConvexIndicatorFunction, param={'D': np.inf})
FpQ  = F + indQ
xs = FpQ.stationary_point();  # x^* is a solution

# (2) Set up the starting points
tx0 = problem.set_initial_point() # this is \tilde{x}^0
x1  = problem.set_initial_point() # this is x^1

# (3) Run the algorithm
tx1,_,_     = proximal_step(x1 - gamma * F.gradient(tx0),indQ,1); # this is \tilde{x}^1
x2,_,_      = proximal_step(x1 - gamma * F.gradient(tx1),indQ,1); # this is x^2
tx2,_,_     = proximal_step(x2 - gamma * F.gradient(tx1),indQ,1); # this is \tilde{x}^2
x3,_,_      = proximal_step(x2 - gamma * F.gradient(tx2),indQ,1); # this is x^3

#  define the expressions (recall that our objective is to verify that
#  expression2 <= expression1 for all F and sequence generated by the
#  past extragradient method.
p2 = (x2-x1)**2 + (x2-x1-2*gamma*(F.gradient(x2)-F.gradient(tx1)))**2;
p3 = (x3-x2)**2 + (x3-x2-2*gamma*(F.gradient(x3)-F.gradient(tx2)))**2;
A3 = A2 + 1/8;
expression1 = ( x2-xs )**2 + 1/16 * (tx1 - tx0)**2 + A2 * p2;
expression2 = ( x3-xs )**2 + 1/16 * (tx2 - tx1)**2 + A3 * p3;

# (4) Set up the performance measure 
expression_to_verify = expression2 - expression1;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if expression2 - expression1 <= 0 for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value # this should be close to zero

Did PEPit verify the potential (within prescribed numerical precision)? True  	


5.100768163401881e-08